#Load flickr30k_dataset

downgrade tensorflow to 2.10.0


In [ ]:
!pip uninstall tensorflow -y


In [ ]:
!pip install tensorflow==2.10.0

In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
from tensorflow.keras.layers import Input , Dense , Embedding , LSTM , Dropout , add
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.xception import Xception
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import TextVectorization
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from nltk.stem.snowball import stopwords
from tensorflow.keras.models import Model
from matplotlib import image as mpimg
import matplotlib.pyplot as plt
#import tensorflow_text as text
import tensorflow_hub as hub
import tensorflow as tf
from PIL import Image
import numpy as np
import seaborn
import random
import pickle
#import spacy
import nltk
import gzip
import cv2
import os
import re
import json

In [ ]:
from google.colab import files
files.upload()  # Select the kaggle.json file you downloaded


In [ ]:
!mkdir ~/.kaggle #Creates a folder to store your credentials.
!mv kaggle.json ~/.kaggle/ #Moves your Kaggle API credentials to the correct location.
!chmod 600 ~/.kaggle/kaggle.json #Protects your credentials from being accessed by anyone else.


In [ ]:
!kaggle datasets download -d adityajn105/flickr8k


In [ ]:
!unzip /content/flickr8k.zip -d /content/flick8k_images


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive/

In [ ]:
!mkdir -p /content/drive/MyDrive/flick8k_images
!cp -vr /content/flick8k_images /content/drive/MyDrive/flick8k_images #Clone dataset to your drive



#Read file & make dictionary *function*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
"""
import os
os.chdir("/content/drive/MyDrive/ai_model_2024/image_captioning")
"""

In [ ]:
"""
import csv

# Reading the file
def read_file(path):
    with open(path, 'r') as file:
        return file.read().splitlines()  # `splitlines()` is a better option here

test = read_file("/content/sample_data/test.txt")
print(test[:5])  # Print the first 5 lines to check
"""

In [ ]:
def get_data_dictionary(data):
    descriptions = {}
    for line in data:
        try:
            image_name, caption = line.split(',', 1)  # `split` only on the first comma
            image_name = image_name.strip()  # Remove any extra spaces from image name
            caption = caption.strip()  # Clean up the caption text

            # Add the caption to the list of captions for the corresponding image
            if image_name in descriptions:
                descriptions[image_name].append(caption)
            else:
                descriptions[image_name] = [caption]
        except ValueError:
            print(f"Skipping malformed line: {line}")  # If there's an error in the line format
    return descriptions

In [ ]:
import csv

# Reading the file
def read_file(path):
    with open(path, 'r') as file:
        return file.read().splitlines()  # `splitlines()` is a better option here

# Converting image_captions data into a dict where keys = images and values = captions
def get_data_dictionary(data):
    descriptions = {}
    for line in data:
        try:
            image_name, caption = line.split(',', 1)  # `split` only on the first comma
            image_name = image_name.strip()  # Remove any extra spaces from image name
            caption = caption.strip()  # Clean up the caption text

            # Add the caption to the list of captions for the corresponding image
            if image_name in descriptions:
                descriptions[image_name].append(caption)
            else:
                descriptions[image_name] = [caption]
        except ValueError:
            print(f"Skipping malformed line: {line}")  # If there's an error in the line format
    return descriptions

In [ ]:

data = read_file("/content/drive/MyDrive/ai_model_2024/image_captioning/captions.txt") # caption path
print(data[:5])  # Print the first 5 lines to check
descriptions = get_data_dictionary(data)
#print(list(descriptions.items())[:5])  # Print the first 5 image-caption pairs

# Print a few entries to verify
for image_name, captions in list(descriptions.items())[:5]:
    print(f"Image: {image_name}")
    for i, caption in enumerate(captions):
        print(f"  Caption {i+1}: {caption}")

#Text Preprocessing Functions:

In [ ]:
#print(remove_punc("KFC5$%&@(#())"))
#print(to_lower_case("KFC IS tHE BEST Fast food chain in the world"))
#

In [ ]:
import nltk
nltk.download('stopwords')


In [ ]:


import re
from nltk.corpus import stopwords

def remove_punc(text):
    return re.sub(r'[^\w\s]', '', text)

def to_lower_case(text):
    return text.lower()

stopwords_list = stopwords.words('english')
def remove_stopwords(text):
    text_words = [word for word in text.split() if (word not in stopwords_list) and (len(word) > 2)]
    text = " ".join(text_words)
    return text

def remove_numbers(text):
    return re.sub(r'[0-9]', '', text)

def remove_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text).strip()

# Gathering all the text cleaning steps in one function
def clean_text(text):
    text = remove_punc(text)
    text = to_lower_case(text)
    text = remove_stopwords(text)
    text = remove_numbers(text)
    text = remove_multiple_spaces(text)
    return text



#Cleaning Captions(Optional Could skip if u already have clean txt fie)

In [ ]:
# Using predefined text preprocessing functions to clean the captions text
def clean_captions(descriptions):
    for image in descriptions.keys():
        for index, caption in enumerate(descriptions[image]):
            descriptions[image][index] = clean_text(caption)
    return descriptions


In [ ]:
# Example usage
descriptions = get_data_dictionary(data)
descriptions = clean_captions(descriptions)

#Saving the cleaned data(Optional Could skip if u already have clean txt fie)


In [ ]:
import os

# Saving data dictionary into an external file
def write_file(path, data):
    lines = []
    for image in data.keys():
        for caption in data[image]:
            lines.append(image + '\t' + caption)
    lines = '\n'.join(lines)

    with open(path, 'w') as file:
        file.write(lines)


In [ ]:
"""
test = "/content/drive/MyDrive/ai_model_2024/image captioning/Dataset/flick8k_images/cleaned_data.txt"
clean_data = read_file(test)
print(clean_data[:5])  # Print the first 5 lines to check
for line in clean_data[1:21]:  # Skip header
    print(line.split('\t'))
"""

In [ ]:
path = os.path.join("/content/drive/MyDrive/flick8k_images", 'cleaned_data.txt')
write_file(path, descriptions)

#Extract image featured using VGG16 pre-train model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os
import numpy as np
import pickle
import gc
from PIL import Image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model

# Importing ResNet50 model without the output layer
features_extractor = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
features_extractor = Model(inputs=features_extractor.inputs, outputs=features_extractor.layers[-1].output)

# Path to the folder containing images
images_path = "/content/drive/MyDrive/flick8k_images/Images"
images_names = sorted(os.listdir(images_path))

# Directory to save the features
features_save_dir = '/content/drive/MyDrive/flick8k_images/features'
os.makedirs(features_save_dir, exist_ok=True)
features_save_file = os.path.join(features_save_dir, 'features.bin')

# Function to process a single image with error handling
def process_image(img_path):
    try:
        img = Image.open(img_path)
        img = img.resize((224, 224))  # Resize as required for ResNet50
        img = np.expand_dims(img, axis=0)
        img = np.array(img) / 255.0  # Normalize to [0,1] as expected by ResNet50
        return img
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return None

# Function to extract features in batches
def preprocess_image(model, images_path, images_list, batch_size=1):
    with open(features_save_file, 'ab') as f:  # 'ab' mode: append binary
        total_processed = 0  # Counter for processed images

        for i in range(0, len(images_list), batch_size):
            batch_images = images_list[i:i + batch_size]
            batch_features = {}

            for img_name in batch_images:
                img_path = os.path.join(images_path, img_name)
                image = process_image(img_path)  # Process the image
                if image is not None:  # Only process valid images
                    feature = model.predict(image)  # Extract feature from model
                    batch_features[img_name] = feature  # Store in dictionary
                    total_processed += 1  # Increment the counter

            # Save the batch of features to file
            pickle.dump(batch_features, f)

            # Print the number of processed images
            print(f"Processed {total_processed} images so far.")

            # Clear memory after each batch
            del batch_images, batch_features
            gc.collect()  # Explicitly trigger garbage collection to free memory


In [ ]:
# Call the function
preprocess_image(features_extractor, images_path, images_names)


In [ ]:
import pickle

# Path to the features file
features_save_file = '/content/drive/MyDrive/flick8k_images/features/features.bin'

# Function to load features from a binary file
def load_features(file_path):
    all_features = {}
    with open(file_path, 'rb') as f:
        while True:
            try:
                batch_features = pickle.load(f)  # Load batch of features
                all_features.update(batch_features)  # Update the dictionary with the batch
            except EOFError:
                break  # End of file reached
    return all_features

# Load features
features = load_features(features_save_file)

# Display keys and a sample feature vector to verify
print(f"Total images processed: {len(features)}")
for img_name, feature_vector in features.items():
    print(f"Image: {img_name}, Feature shape: {feature_vector.shape}")
    print(f"Sample feature vector for {img_name}:\n {feature_vector}")
    break  # Print for only one image as an example


#Preprocessing the Captions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from collections import defaultdict

def load_tokens(path, images):
    tokens = defaultdict(list)  # Using defaultdict to simplify appending
    try:
        lines = read_file(path)
        for line in lines:
            img, caption = line.split('\t')
            if img in images:
                tokens[img].append("<start> " + caption + " <end>")
    except Exception as e:
        print(f"Error while loading tokens: {e}")
    return tokens

In [ ]:
def read_file(path):
    try:
        with open(path, 'r') as file:
            return file.read().splitlines()  # `splitlines()` is a better option here
    except Exception as e:
        print(f"Error while reading file: {e}")
        return []


#Splitting the Dataset

In [ ]:
import os
from sklearn.model_selection import train_test_split
import pickle

# Function to read file and list all available images
def list_images(path):
    all_images = []
    # Assuming `read_file` is a function that reads the file into lines
    lines = read_file(path)

    # Check if the first line is a header
    if 'image' in lines[0].lower():
        lines = lines[1:]  # Skip the header

    for line in lines:
        img, caption = line.split('\t')
        if img not in all_images:
            all_images.append(img)
    return all_images


In [ ]:
# Path to the cleaned data
cleaned_data_path = r"/content/drive/MyDrive/flick8k_images/cleaned_data.txt"

# Listing all available images
all_images_list = list_images(cleaned_data_path)

# Check if we have images before proceeding
if len(all_images_list) == 0:
    print("No images found in the provided cleaned data file.")
else:
    print(f"Total images found: {len(all_images_list)}")

    # Proceed to split the dataset only if there are images
    training_images, testing_images = train_test_split(all_images_list, test_size=0.1, shuffle=True, random_state=42)
    cross_validation_images, testing_images = train_test_split(testing_images, test_size=0.5, shuffle=True, random_state=42)

    # Filter features to only include those that are in training_images, testing_images, and cross_validation_images
    training_features = {img: features[img] for img in training_images if img in features}
    testing_features = {img: features[img] for img in testing_images if img in features}
    cross_validation_features = {img: features[img] for img in cross_validation_images if img in features}

    # Loading training images captions dict
    training_tokens = load_tokens(r"/content/drive/MyDrive/flick8k_images/cleaned_data.txt", training_images)
    cross_validation_tokens = load_tokens(r"/content/drive/MyDrive/flick8k_images/cleaned_data.txtt", cross_validation_images)
    testing_tokens  = load_tokens(r"/content/drive/MyDrive/flick8k_images/cleaned_data.txt", testing_images)

    # Now you have training_tokens and features split correctly
    print(f"Training images: {len(training_images)}, Features: {len(training_features)}")
    print(f"Testing images: {len(testing_images)}, Features: {len(testing_features)}")
    print(f"Cross-validation images: {len(cross_validation_images)}, Features: {len(cross_validation_features)}")


#Vectorizing the Captions

In [ ]:
# extracting all captions into one list
def fetch_captions(tokens) :
  captions = []
  for caps in tokens.values() :
    [captions.append(cap) for cap in caps]
  return captions

In [ ]:
training_captions = fetch_captions(training_tokens)
validation_captions = fetch_captions(cross_validation_tokens)
testing_captions = fetch_captions(testing_tokens)

In [ ]:
# Function to get max_length and full_text for a given token dictionary
def analyze_captions(tokens):
    captions = fetch_captions(tokens)  # Use your function to fetch captions from tokens
    sentences_length = [len(caption.split()) for caption in captions]
    max_length = max(sentences_length)  # Find the maximum caption length
    full_text = ' '.join(captions)      # Concatenate all captions into a single string
    return max_length, full_text

# Analyze training, validation, and test captions
train_max_length, train_full_text = analyze_captions(training_tokens)
val_max_length, val_full_text = analyze_captions(cross_validation_tokens)
test_max_length, test_full_text = analyze_captions(testing_tokens)

# Display results for each set
print(f"Training set max length: {train_max_length}")
print(f"Validation set max length: {val_max_length}")
print(f"Test set max length: {test_max_length}")


In [ ]:
# Create a TextVectorization layer and adapt it on training captions only
text_dataset_train = tf.data.Dataset.from_tensor_slices(training_captions)
vectorize_layer = TextVectorization(output_mode='int', output_sequence_length=train_max_length)
vectorize_layer.adapt(text_dataset_train)

# Apply vectorization to all three datasets
training_captions_vectorized = vectorize_layer(training_captions)
validation_captions_vectorized = vectorize_layer(validation_captions)
testing_captions_vectorized = vectorize_layer(testing_captions)

print("Vocabulary size:", vectorize_layer.vocabulary_size())
print("Example vectorized caption:", training_captions_vectorized[0])

# After adapting the vectorize_layer on the training data
vocab = vectorize_layer.get_vocabulary()  # List of all vocabulary tokens
vocab_size = len(vocab)  # Size of the vocabulary

# Now you have both as variables
print("Vocabulary:", vocab)
print("Vocabulary Size:", vocab_size)


In [ ]:

# Define the save path in Google Drive
save_path = '/content/drive/MyDrive/flick8k_images'

#Pickle the config and weights then Save the pickle file to the save path
pickle.dump({'config': vectorize_layer.get_config(),
             'weights': vectorize_layer.get_weights()},
            open(save_path, "wb"))

In [ ]:
# tokenizing captions and saving it back to dict where keys:images and values:sequences
training_images_sequences = {}
i = 0
for img , captions in training_tokens.items():
    training_images_sequences[img] = []
    for caption in captions :
        sequence =  vectorize_layer(tf.constant([caption])).numpy().tolist()[0]
        training_images_sequences[img].append(sequence)

In [ ]:
validate_images_sequences = {}
i = 0
for img , captions in cross_validation_tokens.items():
    validate_images_sequences[img] = []
    for caption in captions :
        sequence =  vectorize_layer(tf.constant([caption])).numpy().tolist()[0]
        validate_images_sequences[img].append(sequence)

In [ ]:

testing_images_sequences = {}
i = 0
for img , captions in testing_tokens.items():
    testing_images_sequences[img] = []
    for caption in captions :
        sequence =  vectorize_layer(tf.constant([caption])).numpy().tolist()[0]
        testing_images_sequences[img].append(sequence)

In [ ]:
with open('/content/drive/MyDrive/flick8k_images/training_images_sequenceslastest.json', 'w') as f:
#     # Serialize the dictionary to JSON and write it to the file
     json.dump(training_images_sequences, f)

In [ ]:
with open('/content/drive/MyDrive/flick8k_images/validate_images_sequenceslastest.json', 'w') as f:
#     # Serialize the dictionary to JSON and write it to the file
     json.dump(validate_images_sequences, f)

In [ ]:
with open('/content/drive/MyDrive/flick8k_images/testing_images_sequenceslastest.json', 'w') as f:
#     # Serialize the dictionary to JSON and write it to the file
     json.dump(testing_images_sequences, f)

In [ ]:
print(captions)
print(train_max_length)
print(vocab)
print(training_images_sequences)
print(validate_images_sequences)

#8-Building data generator

In [ ]:
import tensorflow as tf
import numpy as np
import json
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gc  # Import garbage collection

# Parameters
BATCH_SIZE = 16  # Reduced batch size to save memory
BUFFER_SIZE = 1000
MAX_LENGTH = train_max_length
VOCAB_SIZE = vocab_size  # Set your actual vocab size here

# Load your data
with open('/content/drive/MyDrive/flick8k_images/training_images_sequenceslastest.json', 'r') as f:
    training_images_sequences = json.load(f)

with open('/content/drive/MyDrive/flick8k_images/validate_images_sequenceslastest.json', 'r') as f:
    validate_images_sequences = json.load(f)

def data_generator(sequences, features, vocab_size, max_length, batch_size):  # Reduced batch size
    while True:
        input_1, input_2, output = [], [], []
        for img, caps in sequences.items():
            feature = features[img].squeeze().astype(np.float32)  # Ensure correct feature shape
            for cap in caps:
                caption = pad_sequences([cap], maxlen=max_length, padding='post')[0].astype(np.int32)  # Correct caption shape

                # Generate targets by shifting the caption sequence
                for i in range(1, len(caption)):
                    input_2.append(caption[:i])
                    output.append(caption[i])  # Append integer label instead of one-hot encoded
                    input_1.append(feature)

                # Yield batch when full
                if len(input_1) >= batch_size:
                    input_1_batch = np.array(input_1[:batch_size])
                    input_2_batch = pad_sequences(input_2[:batch_size], maxlen=max_length, padding='post')
                    output_batch = np.array(output[:batch_size])

                    # Yield a tuple matching the output_signature
                    yield ((input_1_batch, input_2_batch), output_batch)
                    input_1, input_2, output = [], [], []  # Reset batch

        # Optional: clear variables to save memory
        del input_1, input_2, output
        gc.collect()

# Wrapping the generator in a tf.data.Dataset without batching here
dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(training_images_sequences, training_features, VOCAB_SIZE, MAX_LENGTH, BATCH_SIZE),
    output_signature=(
        (tf.TensorSpec(shape=(None, 7, 7, 2048), dtype=tf.float32),  # Image feature shape
         tf.TensorSpec(shape=(None, MAX_LENGTH), dtype=tf.int32)),    # Caption shape
        tf.TensorSpec(shape=(None,), dtype=tf.int32)                  # Target integer label shape
    )
).shuffle(BUFFER_SIZE).prefetch(tf.data.AUTOTUNE)


# Create the validation dataset
validate_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(validate_images_sequences, cross_validation_features, VOCAB_SIZE, MAX_LENGTH, BATCH_SIZE),
    output_signature=(
        (tf.TensorSpec(shape=(None, 7, 7, 2048), dtype=tf.float32),  # Image feature shape
         tf.TensorSpec(shape=(None, MAX_LENGTH), dtype=tf.int32)),    # Caption shape
        tf.TensorSpec(shape=(None,), dtype=tf.int32)                  # Target integer label shape
    )
).prefetch(tf.data.AUTOTUNE)  # You can also shuffle if needed

In [ ]:
# Test the generator
for (img_batch, cap_batch), target_batch in dataset.take(1):
    print("Image batch shape:", img_batch[0].shape)  # Should print (BATCH_SIZE, 7, 7, 2048)
    print("Caption batch shape:", cap_batch.shape)    # Should print (BATCH_SIZE, max_length)
    print("Target batch shape:", target_batch.shape)   # Should print (BATCH_SIZE, vocab_size)
    break


In [ ]:
def check_keys_match(training_images_sequences, training_features):
    # Get the keys from both dictionaries
    token_keys = set(training_tokens.keys())
    feature_keys = set(training_features.keys())

    # Check if they match
    if token_keys == feature_keys:
        print("All keys match between training_tokens and features.")
        return True
    else:
        print("Keys do not match!")

        # Find keys that are in training_tokens but not in features
        missing_in_features = token_keys - feature_keys
        if missing_in_features:
            print(f"Missing in features: {missing_in_features}")

        # Find keys that are in features but not in training_tokens
        missing_in_tokens = feature_keys - token_keys
        if missing_in_tokens:
            print(f"Missing in training_tokens: {missing_in_tokens}")

        return False

check_keys_match(training_tokens, training_features)

#Building the Model

Find max_size vocab size

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, LSTM, Embedding, add


In [ ]:

no_of_features = 100352

from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Add, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization
import tensorflow as tf

def build_transformer_model(no_of_features, max_length, vocab_size, learning_rate):
    # Image feature input
    image_input = Input(shape=(7, 7, 2048))
    image_flat = Flatten()(image_input)
    image_features = Dense(256, activation='relu')(image_flat)

    # Caption input (tokenized caption sequence)
    caption_input = Input(shape=(max_length,))
    caption_embed = Embedding(vocab_size + 2, 300)(caption_input)

    # Transformer Encoder Layer
    transformer_output = MultiHeadAttention(num_heads=4, key_dim=256)(caption_embed, caption_embed)
    transformer_output = LayerNormalization()(transformer_output)
    transformer_output = Dense(256, activation='relu')(transformer_output)

    # Combine image and caption features
    combined = Add()([image_features, tf.reduce_mean(transformer_output, axis=1)])  # Using mean pooling
    dense_1 = Dense(256, activation='relu')(combined)
    output = Dense(vocab_size, activation='softmax')(dense_1)

    # Build and compile model
    model = Model(inputs=[image_input, caption_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize the Transformer-based model
captioning_model = build_transformer_model(no_of_features, train_max_length, vocab_size, 0.001)


In [ ]:
# prompt: plot model

plot_model(captioning_model , show_shapes = True)

#Training the Model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

tf.debugging.enable_check_numerics()
# Create a timestamped filename for saving the best model
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
checkpoint_filename = f'/content/drive/MyDrive/flick8k_images/best_model/{timestamp}.h5'

# Set up ModelCheckpoint to save the best model
checkpoint_callback = ModelCheckpoint(
    checkpoint_filename,
    monitor='val_loss',  # Monitor validation loss
    save_best_only=True,
    mode='min',
    verbose=1
)

EPOCHS = 20
# Fit the model
history = captioning_model.fit(
    dataset,
    epochs=EPOCHS,
    steps_per_epoch=len(training_images_sequences) // BATCH_SIZE,
    validation_data=validate_dataset,  # Validation dataset
    validation_steps=len(validate_images_sequences) // BATCH_SIZE,  # Validation steps
    verbose=1,
    callbacks=[checkpoint_callback]  # Include the checkpoint callback
)

# Save training history
with open(f'/content/drive/MyDrive/flick8k_images/training_history_{timestamp}.json', 'w') as f:
    json.dump(history.history, f)


In [ ]:
captioning_model.save('/content/drive/MyDrive/flick8k_images/best_model/next_captioning_model.h5')

In [ ]:
!pip freeze > requirements.txt


In [ ]:
def write_list_to_file(input_list, file_name):
    with open(file_name, "w") as file:
        for item in input_list:
            file.write(str(item) + "\n")

In [ ]:
print("Sequence:", sequence)
print("Type of elements:", [type(s) for s in sequence])


In [ ]:
# loading txt file into list
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

In [ ]:
model_loss = read_file_to_list(r"E:\Khaled\Data\Projects\Image Caption Generator - GPU\saved_models\model_loss.txt")

#Loading model


In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model

# Load the ResNet50 model for feature extraction
features_extractor = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
features_extractor = Model(inputs=features_extractor.inputs, outputs=features_extractor.layers[-1].output)

In [ ]:
max_length = 22
img_path = "/content/car.jpg"

# Function to process a single image
def get_features_from_image(img_path,model):
    try:
        img = Image.open(img_path)
        img = img.resize((224, 224))  # Resize as required for ResNet50
        img = np.expand_dims(img, axis=0)
        img = np.array(img) / 255.0  # Normalize to [0,1]
        features = model.predict(img)
        return features
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return None

def get_word(index,vocab) :
  word = vocab[index]
  return word

def get_caption(path, features_extractor, vectorize_layer, captioning_model, max_length=22):
    my_features = get_features_from_image(path, features_extractor)
    caption = 'start'
    for i in range(max_length):
        sequenced_caption = vectorize_layer(tf.constant([caption])).numpy().tolist()
        padded_sequenced_caption = pad_sequences(sequenced_caption, maxlen=max_length, padding='post')[0]
        padded_sequenced_caption = np.expand_dims(padded_sequenced_caption, axis=0)

        output = captioning_model.predict([my_features, padded_sequenced_caption])
        index = np.argmax(output[0])

        # Check if predicted word is <end>
        if index == 2:  # 2 is the index for 'end'
            break
        else:
            current_word = get_word(index, vocabulary)
            caption += ' ' + current_word
            print(f"Predicted word: {current_word}")

    # Remove 'start' and 'end' tokens for a clean caption
    return caption.replace('start ', '').replace(' end', '')


def get_caption_show_image(image_path=None) :
    if image_path is not None :
        image_path = image_path
    else :
        images_names = read_file(r"E:\Khaled\Data\Projects\Image Caption Generator - GPU\testing_images_list.txt")
        image_index = random.randint(0,len(images_names))
        image_path = os.path.join(r"E:\Khaled\Data\Projects\Image Caption Generator - GPU\flickr30k_images",images_names[image_index])

    caption = get_caption(image_path,features_extractor,vectorize_layer,captioning_model)[8:-5]
    print(caption)
    image = plt.imread(image_path)
    fig,ax = plt.subplots()
    ax.imshow(image)
    ax.axis('off')

    plt.show()

In [ ]:
get_caption_show_image(img_path)

In [ ]:
print("Vocabulary:", vocabulary)
print("Index of '<start>':", vocabulary.index('<start>'))
print("Index of '<end>':", vocabulary.index('<end>'))
